# AnyoneAI - Sprint Project 01
> E-Commerce Data Pipeline

In this notebook, we will first execute all the components of our ELT pipeline and then plot visualizations with the resulting information.


In [1]:
# Importing the necessary libraries
from pandas import DataFrame
from sqlalchemy import create_engine
from typing import Dict
from pathlib import Path

from src.transform import QueryEnum
from src import config
from src.transform import run_queries
from src.extract import extract
from src.load import load
from src.plots import (
    plot_freight_value_weight_relationship,
    plot_global_amount_order_status,
    plot_real_vs_predicted_delivered_time,
    plot_revenue_by_month_year,
    plot_revenue_per_state,
    plot_top_10_least_revenue_categories,
    plot_top_10_revenue_categories,
    plot_delivery_date_difference,
    plot_order_amount_per_day_with_holidays,
)

c:\Users\jcpmp\OneDrive\Desktop\Anyone AI\Sprint 1\E-Commerce-Data-Pipeline-main\dataset


ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
# Create the database sql file
Path(config.SQLITE_BD_ABSOLUTE_PATH).touch()

# Create the database connection
db_path = Path(config.SQLITE_BD_ABSOLUTE_PATH).resolve().as_posix()
ENGINE = create_engine(f"sqlite:///{db_path}", echo=False)
print(ENGINE)

### 1. Extract

Here you will be extracting the information from different sources.

In [ ]:
csv_folder = config.DATASET_ROOT_PATH
public_holidays_url = config.PUBLIC_HOLIDAYS_URL

# 1. Get the mapping of the csv files to the table names.
csv_table_mapping = config.get_csv_to_table_mapping()

# 2. Extract the data from the csv files, holidays and load them into the dataframes.
csv_dataframes = extract(csv_folder, csv_table_mapping, public_holidays_url)

### 2. Load

Using the diferent datasets provided to you, you will store everything into our Data Warehouse.

In [ ]:
load(data_frames=csv_dataframes, database=ENGINE)

### 3. Transform

Here you will be transforming the different queries into tables. There are two sections, one with the revenue-related queries, and another with the delivery-related queries.

##### 3.1 Revenue queries

In [ ]:
query_results: Dict[str, DataFrame] = run_queries(database=ENGINE)

In [ ]:
# Transforming the revenue_by_month_year query to a table
revenue_by_month_year = query_results[QueryEnum.REVENUE_BY_MONTH_YEAR.value]
revenue_by_month_year

In [ ]:
# Transforming the top_10_revenue_categories query to a table
top_10_revenue_categories = query_results[QueryEnum.TOP_10_REVENUE_CATEGORIES.value]
top_10_revenue_categories

In [ ]:
# Transforming the top_10_least_revenue_categories query to a table
top_10_least_revenue_categories = query_results[
    QueryEnum.TOP_10_LEAST_REVENUE_CATEGORIES.value
]
top_10_least_revenue_categories

In [ ]:
# Transforming the revenue_per_state query to a table
revenue_per_state = query_results[QueryEnum.REVENUE_PER_STATE.value]
revenue_per_state

##### 3.2 Delivery queries

In [ ]:
# Transforming the delivery_date_difference query to a table
delivery_date_difference = query_results[QueryEnum.DELIVERY_DATE_DIFFERECE.value]
delivery_date_difference

In [ ]:
# Transforming the real vs predicted delivered time query to a table per month and year
real_vs_estimated_delivered_time = query_results[
    QueryEnum.REAL_VS_ESTIMATED_DELIVERED_TIME.value
]
real_vs_estimated_delivered_time

In [ ]:
# Transforming the global ammount of order status query to a table
global_ammount_order_status = query_results[QueryEnum.GLOBAL_AMOUNT_ORDER_STATUS.value]
global_ammount_order_status

##### 3.3 Pandas transformations

In [ ]:
orders_per_day_and_holidays = query_results[
    QueryEnum.ORDERS_PER_DAY_AND_HOLIDAYS_2017.value
]
orders_per_day_and_holidays

In [ ]:
orders_per_day_and_holidays[orders_per_day_and_holidays["holiday"] == True]

In [ ]:
freight_value_weight_relationship = query_results[
    QueryEnum.GET_FREIGHT_VALUE_WEIGHT_RELATIONSHIP.value
]
freight_value_weight_relationship

### 4. Plotting the query tables

Now, we will create plots from the different queries

In [ ]:
plot_revenue_by_month_year(revenue_by_month_year, 2018)

In [ ]:
plot_top_10_revenue_categories(top_10_revenue_categories)

In [ ]:
plot_top_10_least_revenue_categories(top_10_least_revenue_categories)

In [ ]:
plot_revenue_per_state(revenue_per_state)

In [ ]:
plot_delivery_date_difference(delivery_date_difference)

In [ ]:
plot_real_vs_predicted_delivered_time(real_vs_estimated_delivered_time, year=2017)

In [ ]:
plot_global_amount_order_status(global_ammount_order_status)

In [ ]:
plot_order_amount_per_day_with_holidays(orders_per_day_and_holidays)

The Orders per Day and Holidays plot you will code in src/plots.py has to be similar to this:

![The plot you will code will be similar to this:](images/orders_per_day_and_holidays.png "Orders per Day and Holidays")

4.1 Do you see any relationship between the Brazilian public holidays of 2017 and that year's purchases?

*We can see an increment in purchases after holidays. It's something little, sutil. Between 2017-11 and 2018-01 we can see a big increment and this is normal for christmas holiday.* 

In [ ]:
plot_freight_value_weight_relationship(freight_value_weight_relationship)

The Freight Value and Weight Relationship plot you will code in src/plots.py has to be similar to this:

![The plot you will code will be similar to this:](images/freight_value_weight_relationship.png "Freight Value and Weight Relationship")

4.2 Do you see any correlation between the freight value and the product weight?

*There is an obvious relationship between product weight and freight value. For heaviest products, biggest costs in freight values, with some exceptions.*